In [1]:
!pip install jsonlines

In [2]:
import os
import jsonlines
import random
import pandas as pd
from typing import List
from typing import Tuple
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
file_path = '/content/drive/MyDrive/Colab Notebooks/dataset/trn.json'

df = pd.read_json(file_path, lines=True)

KeyboardInterrupt: 

In [ ]:
num_rows, num_columns = df.shape
print(f"Total de linhas: {num_rows}")
print(f"Total de colunas: {num_columns}")

In [ ]:
print("\nPrimeiras linhas do dataset:")
print(df.head())

In [ ]:
def load_dataset(input_file: str) -> List[dict]:
    """
    Carrega o dataset a partir de um arquivo JSONLines.

    Args:
        input_file (str): Caminho do arquivo JSONLines de entrada.

    Returns:
        List[dict]: Lista de dicionários com os dados do JSONLines.
    """
    try:
        with jsonlines.open(input_file, mode='r') as reader:
            return [line for line in reader]
    except Exception as e:
        raise ValueError(f"Erro ao carregar o arquivo: {e}")

def validate_inputs(input_file: str, fraction: float) -> None:
    """
    Valida os inputs para a função create_sample_dataset.
    """
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"O arquivo {input_file} não foi encontrado.")
    if not (0 < fraction <= 1):
        raise ValueError("A fração deve estar no intervalo (0, 1].")

In [ ]:
def filter_empty_entries(data: pd.DataFrame) -> pd.DataFrame:
    """
    Remove entradas vazias do DataFrame.

    Args:
        data (pd.DataFrame): DataFrame contendo as colunas 'title' e 'content'.

    Returns:
        pd.DataFrame: DataFrame filtrado sem entradas vazias.
    """
    print(f"Linhas antes da filtragem: {len(data)}")
    filtered_data = data.dropna(subset=['title', 'content']).query("title != '' and content != ''")
    print(f"Linhas após a filtragem: {len(filtered_data)}")
    return filtered_data

In [ ]:
def create_sample_dataset(input_file: str, output_file: str, fraction: float, seed: int) -> Tuple[int, int]:
    """
    Extrai uma fração aleatória do dataset, mantém 'title' e 'content',
    remove entradas vazias e salva em um novo arquivo.

    Args:
        input_file (str): Caminho do arquivo JSONLines de entrada.
        output_file (str): Caminho do arquivo JSONLines de saída.
        fraction (float): Fração do dataset a ser extraída.
        seed (int): Semente para garantir reprodutibilidade.

    Returns:
        Tuple[int, int]: Total de linhas no dataset final e total de registros originais.
    """
    try:
        # Validar entradas
        validate_inputs(input_file, fraction)

        # Carregar o dataset
        print("Carregando o dataset...")
        data = load_dataset(input_file)

        if not data:
            print("O arquivo de entrada está vazio.")
            return 0, 0

        # Configurar a semente para seleção reprodutível
        random.seed(seed)

        # Selecionar amostra
        total_samples = len(data)
        num_samples = max(1, int(total_samples * fraction))
        print(f"Total de registros no arquivo original: {total_samples}")
        print(f"Número de registros selecionados: {num_samples}")
        sampled_data = random.sample(data, num_samples)

        # Processar dados
        print("Processando os dados...")
        processed_data = [
            {"title": entry.get("title", ""), "content": entry.get("content", "")}
            for entry in sampled_data
        ]

        # Converter para DataFrame
        df = pd.DataFrame(processed_data)

        # Filtrar entradas vazias
        df = filter_empty_entries(df)

        # Salvar o dataset tratado
        print(f"Salvando o dataset em {output_file}...")
        df.to_json(output_file, orient="records", lines=True, force_ascii=False)
        print(f"Dataset tratado salvo com sucesso em: {output_file}")

        # Retornar a contagem final de linhas
        total_final = len(df)
        return total_final, total_samples

    except Exception as e:
        print(f"Erro durante o processamento: {e}", exc_info=True)
        return 0, 0

In [ ]:
def analyze_dataset(file_path: str) -> None:
    """
    Analisa um arquivo JSONLines para verificar se todas as linhas seguem o formato esperado.

    Args:
        file_path (str): Caminho para o arquivo JSONLines.
    """
    total_lines = 0
    valid_lines = 0
    missing_title = 0
    missing_content = 0
    empty_title = 0
    empty_content = 0

    try:
        print(f"Analisando o arquivo: {file_path}")
        with jsonlines.open(file_path, mode='r') as reader:
            for line in reader:
                total_lines += 1

                # Verificar se as chaves existem
                has_title = "title" in line
                has_content = "content" in line

                if has_title and has_content:
                    # Contar como válido somente se ambas as chaves existirem
                    valid_lines += 1

                    # Verificar se os valores não estão vazios
                    if not line["title"].strip():
                        empty_title += 1
                    if not line["content"].strip():
                        empty_content += 1
                else:
                    # Contar erros de ausência de campos
                    if not has_title:
                        missing_title += 1
                    if not has_content:
                        missing_content += 1

        # Exibir resumo
        print("Resumo da análise do dataset:")
        print(f"Total de linhas: {total_lines}")
        print(f"Linhas válidas: {valid_lines}")
        print(f"Linhas com 'title' ausente: {missing_title}")
        print(f"Linhas com 'content' ausente: {missing_content}")
        print(f"Linhas com 'title' vazio: {empty_title}")
        print(f"Linhas com 'content' vazio: {empty_content}")
        print(f"Linhas inválidas: {total_lines - valid_lines}")

    except Exception as e:
        print(f"Erro ao analisar o dataset: {e}", exc_info=True)

In [ ]:
input_file = "TechChallenge-Fase3/LF-Amazon/trn-folder/trn.json"
output_file = "TechChallenge-Fase3/LF-Amazon/trn-folder/trn_tratado.json"
fraction = 0.4
seed = 42

In [ ]:
analyze_dataset(input_file)

In [ ]:
print("Iniciando o processamento do dataset...")
total_final, total_original = create_sample_dataset(input_file, output_file, fraction, seed)

if total_original > 0:
    final_percentage = (total_final / total_original) * 100
    print(
        f"Processamento concluído. Total de registros no dataset final: {total_final} "
        f"({final_percentage:.2f}% do total original)."
    )
else:
    print("O dataset original estava vazio ou houve um erro no processamento.")

analyze_dataset(output_file)